In [ ]:
#but storing v instead of use and discard

import math
import gc
import sys
import numpy as np
import multiprocessing
from joblib import Parallel,delayed
import time
import numba
from numba import jit,cuda
import timeit

In [ ]:
#function to multiply polynomials: https://www.geeksforgeeks.org/multiply-two-polynomials-2/

def multiply(A, B):
    m = len(A)
    n = len(B)
    prod = [0 for i in range(m+n-1)]
      
    # Multiply two polynomials term by term
      
    # Take ever term of first polynomial
    for i in range(m):
          
        # Multiply the current term of first 
        # polynomial with every term of 
        # second polynomial.
        for j in range(n):
            prod[i + j] += A[i] * B[j]
  
    return prod
#function to add polynomials: https://www.geeksforgeeks.org/program-add-two-polynomials/

def add(A, B):
    m = len(A)
    n = len(B)
    size = max(m, n)
    sum = [0 for i in range(size)]
 
    # Initialize the product polynomial
     
    for i in range(0, m, 1):
        sum[i] = A[i]
 
    # Take ever term of first polynomial
    for i in range(n):
        sum[i] += B[i]
 
    return sum

In [ ]:

#binary sum has been replaced with the xor operation ^
def vectorbinarysum(v1,v2): #sums binary vectors
    if len(v1) != len(v2):
        return
    n = len(v1)
    sum =[]
    for i in range(n):
       sum.append(v1[i]^v2[i])
    return sum

def decimalToBinary(n,x):
    a = bin(n).replace("0b", "")
    return a.zfill(x)

def combinatorial(n,r):
    return math.comb(n,r)
    #return math.factorial(n)/(math.factorial(r)*math.factorial(n-r))

def combsum(m,r): #sums combinatorials from (m,0) to (m,r), gives dimension of rm code
    sum = 0
    for i in range(r+1):
        sum+= combinatorial(m,i)
    return sum
generator = 1 #intialising generator matrix
#fullcode can be y, yy or any other. Each results in a separate case, details below

def code_generator(r,m,fullcode):#generates code based on requirement
    n = 2**m
    basis = [[0 for x in range(n)] for y in range(m)] #initialising the set of boolean funtions v_1 to v_m except unit vector 1
    for i in range(n):
            a = list(decimalToBinary(i,m))
            for j in range(m):
                basis[j][i] = int(a[j])
    k = int(combsum(m,r))
    global generator
    generator = [[0 for x in range(n)] for y in range(m+1)] # initialising generator matrix
    for i in range(n):
        for j in range(1,m+1):
            generator[j][i] = basis[j-1][i]
    for i in range(n):
        generator[0][i]=1
    # Below two functions are to print all combination
    # of size r in an array of size n
    # Used for obtaining row of genereartor matrix
    def printCombination(arr, m, r, n):
        data = [0]*r;
        combinationUtil(arr, data, 0,
                            m - 1, 0, r,n);
        # arr[] ---> Input 
        # Array data[] ---> Temporary array to store current combination
        # start & end ---> Starting and Ending indexes in arr[]
        # index ---> Current index in data[]
        # r ---> Size of a combination to be printed
        # n is size of codeword
    def combinationUtil(arr, data, start,
                            end, index, r,n):
        if (index == r):
            data2 = [1 for x in range(n)]
            for j in range(r):
                for x in range(n):
                    data2[x] *= data[j][x]
            global generator
            generator.append(data2)
            return;
        i = start;
        while(i <= end and end - i + 1 >= r - index):
            data[index] = arr[i];
            combinationUtil(arr, data, i + 1,
                            end, index + 1, r,n);
            i += 1;
    #printcombination and combinationUtil are modifications of code obtained from
    #https://www.geeksforgeeks.org/print-all-possible-combinations-of-r-elements-in-a-given-array-of-size-n/

    
    for i in range(2,r+1):
        printCombination(basis, m, i, n)#i instead of r fixed bug
    k2 = int(combinatorial(m,r))
    if fullcode == 'y': #generates the complete r,m code
        rmcode = [[0 for i in range(n)] for j in range(2**k)]
        coefficient = [1 for i in range(2**k)]
        #coefficeint stores all binary vectors from 0 to 2**k, used to generate all possible linear combinations of the
        #generator matrix
        for i in range(2**k):
            a = list(decimalToBinary(i,k))
            for j in range(k):
                a[j] = int(a[j]) 
            coefficient[i] = a
        for i in range(2**k):
            for j in range(n):
                for t in range(k):
                    rmcode[i][j] = rmcode[i][j]^(coefficient[i][t]*generator[t][j])
        return rmcode
    elif fullcode == 'yy': #find codewords of r,m code not in r-1, m code
        rmcode = [[0 for i in range(n)] for j in range(2**k - 2**(k-k2))]
        coefficient = []
        for i in range(2**k):
            a = list(decimalToBinary(i,k))
            for j in range(k):
                a[j] = int(a[j])
           
            if a[k-k2::k] != [0 for i in range(k2)]:
                coefficient.append(a)
        for i in range(2**k - 2**(k-k2)):
            for j in range(n):
                for t in range(k):
                    rmcode[i][j] =rmcode[i][j]^(coefficient[i][t]*generator[t][j])
        return rmcode 
    elif fullcode == 1: #If you just wish to obtain generator array
        return
    else: #finds the codewords with the basis as all basis vectors of r,m RM code (r,m are arguments passed to the function) 
        #minus the basis vectors of the r-1,m RM code
        rmcode = [[0 for i in range(n)] for j in range(2**k2)]
        print(1,2**k2,sys.getsizeof(rmcode))
        coefficient = [1 for i in range(2**k2)]
        for i in range(2**k2):
            a = list(decimalToBinary(i,k2))
            for j in range(k2):
                a[j] = int(a[j]) 
            coefficient[i] = a
        print(1)
        
        for i in range(2**k2):
            
            for j in range(n):
                for t in range(k2):
                    rmcode[i][j] = rmcode[i][j]^(coefficient[i][t]*generator[t + k - k2][j])
            
        return rmcode

def code_weight(code): #calculating weight of a code
    n =len(code[0])
    weight = [0 for i in range(n+1)]
    for i in range(len(code)):
        sum=0
        for j in range(n):
            sum+= code[i][j]
        weight[sum] +=1
    return weight

def coset(rmcode,vector): #returns modified rmcode
    for j in range(len(rmcode)):
        rmcode[j] = vectorbinarysum(rmcode[j],vector)
    return rmcode
def cosets(rmcode,vectors,i,n1):
    coset = [1 for j in range(n1)]
    for j in range(n1):
        coset[j] = vectorbinarysum(rmcode[j],vectors[i])
    return coset
def coset2(rmcode,vectors):#does not modify argument rmcode
    coset = [1 for j in range(len(rmcode))]
    for j in range(len(rmcode)):
        coset[j] = vectorbinarysum(rmcode[j],vectors)
    return coset
def allcosets(rmcode,vectors):
    n1 = len(rmcode) # number of codeword
    n3 = len(vectors) #number of vectors, each vector will generate a unique coset
    fullcoset = [1 for i in range(n3)]
    for i in range(n3):
        fullcoset[i] = cosets(rmcode,vectors,i,n1)        
    return fullcoset

def squarepolynomial(array):
    return multiply(array,array)

def lemma1(weight):
    enumerator = np.array([])
    for i in range(len(weight)):
        enumerator = add(enumerator,squarepolynomial(weight[i]))
    return enumerator

def binarytodecimal(binary):
    return int(binary,2)


def sumcodes(c1,c2): #returns the code consisting of the sum of codewords of two linear codes
    n1 = len(c1)
    n2 = len(c2)
    codesum = []#TO OPTIMIZE
    for i in range(n1):
        for j in range(n2):
            codesum.append(vectorbinarysum(c1[i],c2[j]))#not worth using bitwise_xor directly here, maybe try in the funuction
    res = [] # from here till return res, function to remove duplicates in a list obtained from
    for i in codesum: #https://www.geeksforgeeks.org/python-ways-to-remove-duplicates-from-list/
        if i not in res:
            res.append(i)
    return res


In [ ]:
# to obtain the weight enumerator of the (r,m) RM code, enter r-2 and m-2 into variables r and m below
r,m = 1,4

In [ ]:
Coo = code_generator(r,m,'y') #Generates whole code as defined in lemma 2

In [ ]:
v_required = code_generator(r+1,m,0) # generates vectors v needed for lemma 2 

In [ ]:
code_generator(r+2,m,1) # to update generator array for later

In [ ]:
Coj = allcosets(Coo,v_required) #as defined in lemma 2
del v_required

In [ ]:
Coo_length = len(Coo)
del Coo
Mo = len(Coj)
print(Mo)
print(int(2**(combinatorial(m,r+2))))#M
gc.collect()
#Cil = np.array([[[[1]*(2**m)]*Coo_length]*len(u_required)]*Mo)

#list of 2d arrays, #parallel compute this
#as defined in lemma 2
#Memory intensive step
#computational time is < 1 second for 3,6 and 4,6 code, but the program crashes here after approximately 15 minutes in case of 
# finding 3,7 codes. For reference, my laptop has 8gb ram

In [ ]:
print(combsum(3,2),combinatorial(7,4))

In [ ]:
#x=[[0 for i in range(128)] for j in range(2**28)]
#print(sys.getsizeof(x))

In [ ]:
k2 = int(combinatorial(m,r+2))
k1 = int(combsum(m,r+2))
n3 = 2**k2 #=len(u_required)
n1 = Coo_length
print(n1,Mo,n3)
weight = []
n = 2**m
global generator
for i in range(n3):
    
    u = [0 for x in range(n)]
    a = list(decimalToBinary(i,k2))
    for x in range(k2):
            a[x] = int(a[x])
    for x in range(n):
            for t in range(k2):
                u[x] = u[x]^(a[t]*generator[t + k1 - k2][x])
    
    #current = time.time()
    
    for j in range(Mo):
        flag = 0
        sum = [0]    #length of Coj       
        
        for k in range(Mo):
            
            Cil = coset2(Coj[k],u)
            sum =add(sum,multiply(code_weight(Cil),code_weight(sumcodes(Cil,Coj[j]))))


        weight.append(sum)
        
    #print(current - time.time())
        
#FOR 2,9 THIS IS 3X slower as compared to 1.8, all that's been changed is np function to user defined function
#19 seconds per loop to get 3,7

In [ ]:
for i in range(len(weight)):
    print(weight[i])

In [ ]:
A_rmcode = lemma1(weight)# required weight enumerator, coefficients of x stored as a list
#first element is coefficien of x^0 and last of x^n

#for i in range(1,len(A_cosets)):
    #A_rmcode = polyadd(A_rmcode,lemma1(A_cosets[i]))


In [ ]:
print('Weight enumerator of the', r+2,',', m+2, 'rm code') #outputs weight enumerator of the code
print(A_rmcode)

In [ ]:
sum = 0
for i in range(len(A_rmcode)):
    sum+= A_rmcode[i]
print(sum,2**(int(combsum(m+2,r+2))))

In [ ]:
#MacWilliam Equations
n = 2**(m+2)
def krawtchouck(k,n,x):
    sum = 0
    for i in range(k+1):
        if x>=i:
            if (n-x)>=(k-i):
                sum += ((-1)**i)*int(combinatorial(x,i))*int(combinatorial(n-x,k-i))
    return int(sum)
#def macwilliamj(n,we,j): #length of code and weight enumerator, j is the subscript of w.e. of dual, division done outside
    #sum = 0
    #for i in range(n+1):
        #sum += we[i]*krawtchouck(j,n,i)
    #return sum NOT IN USE
dualwe = [1 for i in range(n+1)] #weight ditsribution of the dual of the r+2,m+2 code
for j in range(n+1):
    sum = 0
    for i in range(n+1):
        sum+=A_rmcode[i]*krawtchouck(j,n,i)
    dualwe[j] = sum/2**int(combsum(m+2,r+2))
print(dualwe)
sum = 0
for i in range(n+1):
    sum+=dualwe[i]
print(sum)
print(2**int(combsum(m+2,m-r-1)))
print(dualwe == A_rmcode)
for i in range(n+1):
    if i%2!=0:
        print(dualwe[i])
sum = 0 #check2
for i in range(len(dualwe)):
    sum += dualwe[i]
print(sum)
print(2**int(combsum(m+2,m-r-1)))